# BigQuery Market Data Analysis

This notebook pulls `book_ticker` rows for a single symbol/date from BigQuery and runs a quick descriptive analysis.

Default example in this notebook:
- `symbol = BTCUSDT`
- `trade_date = 2026-02-23`
- `dataset = aster`

In [ ]:
import os
from datetime import date

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from dotenv import load_dotenv
from google.cloud import bigquery

load_dotenv()

In [ ]:
# Configuration
PROJECT_ID = os.getenv("ASTER_BQ_PROJECT") or "<YOUR_PROJECT_ID>"
DATASET = os.getenv("ASTER_BQ_DATASET", "aster")
SYMBOL = "BTCUSDT"
TRADE_DATE = date(2026, 2, 23)

if PROJECT_ID == "<YOUR_PROJECT_ID>":
    raise ValueError("Set ASTER_BQ_PROJECT in your environment or edit PROJECT_ID in this notebook.")

PROJECT_ID, DATASET, SYMBOL, TRADE_DATE

In [ ]:
client = bigquery.Client(project=PROJECT_ID)

sql = f"""
SELECT
  ts_unix_ms,
  ts_dt_utc,
  symbol,
  bid_px,
  bid_qty,
  ask_px,
  ask_qty,
  spread,
  mid,
  imbalance,
  weighted_mid,
  date,
  hour,
  minute,
  second
FROM `{PROJECT_ID}.{DATASET}.book_ticker`
WHERE date = @trade_date
  AND symbol = @symbol
ORDER BY ts_unix_ms
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("trade_date", "DATE", TRADE_DATE),
        bigquery.ScalarQueryParameter("symbol", "STRING", SYMBOL),
    ]
)

book_df = client.query(sql, job_config=job_config).to_dataframe(create_bqstorage_client=False)
book_df["ts"] = pd.to_datetime(book_df["ts_unix_ms"], unit="ms", utc=True)
book_df = book_df.sort_values("ts").reset_index(drop=True)

print(f"rows={len(book_df)}")
book_df.head(5)

In [ ]:
book_df[["bid_px", "ask_px", "mid", "spread", "bid_qty", "ask_qty", "imbalance"]].describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])

In [ ]:
sns.set_theme(style="whitegrid")

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
sns.lineplot(data=book_df, x="ts", y="mid", ax=axes[0], linewidth=0.9)
axes[0].set_title(f"{SYMBOL} Mid Price on {TRADE_DATE}")
axes[0].set_ylabel("mid")

sns.lineplot(data=book_df, x="ts", y="spread", ax=axes[1], linewidth=0.9, color="tomato")
axes[1].set_title(f"{SYMBOL} Spread on {TRADE_DATE}")
axes[1].set_ylabel("spread")
axes[1].set_xlabel("UTC timestamp")

plt.tight_layout()
plt.show()

In [ ]:
# Example statsmodels regression: spread_t ~ spread_(t-1)
reg_df = book_df[["spread"]].copy().dropna()
reg_df["spread_lag1"] = reg_df["spread"].shift(1)
reg_df = reg_df.dropna()

X = sm.add_constant(reg_df[["spread_lag1"]])
y = reg_df["spread"]
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
# Optional local export for reuse
out_path = f"./backtest/book_ticker_{SYMBOL}_{TRADE_DATE.isoformat()}.parquet"
book_df.to_parquet(out_path, index=False)
out_path